# Overview and Import
**Recurrent Neural Networks (RNNs) are a type of neural network that are designed to handle sequential data, such as time series or natural language text. They achieve this by using a hidden state that is updated for each time step of the input sequence, allowing the network to maintain a memory of previous inputs.**


**This notebook contains an implementation of an RNN that can be used for language modeling. The self takes in a sequence of characters and outputs the probability distribution over the next character in the sequence. The network is trained on a corpus of text and then used to generate new text that has a similar distribution of characters as the training corpus.**

In [1]:
import numpy as np
import scipy as sp
import os

from RNN import RNN

# Data Prepartion

In [2]:
class DataGenerator:
    """
    A class for generating input and output examples for a character-level language model.
    """

    def __init__(self, path):
        """
        Initialize a DataGenerator object.

        Arg:
            path (str): The path to the text file containing the training data.
        """
        self.path = path

        # Read in data from file and convert to lowercase
        with open(path) as f:
            data = f.read().lower()

        # create list of unique characters in the data
        self.chars = list(set(data))

        # Create dictionaries mapping characters to and from their index in the list of unique characters
        self.char_to_idx = {ch: i for (i, ch) in enumerate(self.chars)}
        self.idx_to_char = {i: ch for (i, ch) in enumerate(self.chars)}

        # Set the size of the vocabulary (i.e. number of unique characters)
        self.vocab_size = len(self.chars)

        # Read in examples from file and covert to lowercase, removing leading/training while space
        with open(path) as f:
            examples = f.readlines()
        self.examples = [x.lower().strip() for x in examples]

    def generate_example(self, idx):
        """
        Generate an input/output example for the language model based on the given index.

        Arg:
            idx (int): The index of the example to generate.

        Returns:
            A tuple containing the input and output arrays for the example.
        """
        example_chars = self.examples[idx]

        # Convert the characters in the example to their corresspoding indices in the list of unique characters
        example_char_idx = [self.char_to_idx[char] for char in example_chars]

        # Add newline character as the first characters in the input array, and as the last characters in the output array
        X = [self.char_to_idx['\n']] + example_char_idx
        Y = example_char_idx + [self.char_to_idx['\n']]

        return np.array(X), np.array(Y)

# RNN Implementation

**The RNN used in this notebook is a basic one-layer RNN. It consists of an input layer, a hidden layer, and an output layer. The input layer takes in a one-hot encoded vector representing a character in the input sequence. This vector is multiplied by a weight matrix  $W_{ax}$ to produce a hidden state vector $a$. The hidden state vector is then passed through a non-linear activation function (in this case, the hyperbolic tangent function) and updated for each time step of the input sequence. The updated hidden state is then multiplied by a weight matrix  $W_{ya}$ to produce the output probability distribution over the next character in the sequence.**

**The RNN is trained using stochastic gradient descent with the cross-entropy loss function. During training, the self takes in a sequence of characters and outputs the probability distribution over the next character. The true next character is then compared to the predicted probability distribution, and the parameters of the network are updated to minimize the cross-entropy loss.**

## Activation Functions
### Softmax Activation Function

**$$\mathrm{softmax}(\mathbf{x})_i = \frac{e^{x_i}}{\sum_{j=1}^n e^{x_j}}
$$**

**The softmax function is commonly used as an activation function in neural networks, particularly in the output layer for classification tasks. Given an input array $x$, the softmax function calculates the probability distribution of each element in the array**

### Tanh Activation
**$$\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$**

**where $x$ is the input to the function. The output of the function is a value between -1 and 1. The tanh activation function is often used in neural networks as an alternative to the sigmoid activation function, as it has a steeper gradient and can better model non-linear relationships in the data.**
****

## Forward propagation:

**During forward propagation, the input sequence is processed through the RNN to generate an output sequence. At each time step, the hidden state and the output are computed using the input, the previous hidden state, and the RNN's parameters.**

**The equations for the forward propagation in a basic RNN are as follows:**

**At time step $t$, the input to the RNN is $x_t$, and the hidden state at time step $t-1$ is $a_{t-1}$. The hidden state at time step $t$ is computed as:**

**$a_t = \tanh(W_{aa} a_{t-1} + W_{ax} x_t + b_a)$**

**where $W_{aa}$ is the weight matrix for the hidden state, $W_{ax}$ is the weight matrix for the input, and $b_a$ is the bias vector for the hidden state.**

**The output at time step $t$ is computed as:**

**$y_t = softmax(W_{ya} a_t + b_y)$**

**where $W_{ya}$ is the weight matrix for the output, and $b_y$ is the bias vector for the output.**
****

## Backward propagation:

**The objective of training an RNN is to minimize the loss between the predicted sequence and the ground truth sequence. Backward propagation calculates the gradients of the loss with respect to the RNN's parameters, which are then used to update the parameters using an optimization algorithm such as Adagrad or Adam.**

**The equations for the backward propagation in a basic RNN are as follows:**

**At time step $t$, the loss with respect to the output $y_t$ is given by:**

**$\frac{\partial L}{\partial y_t} = -\frac{1}{y_{t,i}} \text{ if } i=t_i, \text{ else } 0$**

**where $L$ is the loss function, $y_{t,i}$ is the $i$th element of the output at time step $t$, and $t_i$ is the index of the true label at time step $t$**.

**The loss with respect to the hidden state at time step $t$ is given by:**

**$\frac{\partial L}{\partial a_t} = \frac{\partial L}{\partial y_t} W_{ya} + \frac{\partial L}{\partial h_{t+1}} W_{aa}$**

**where $\frac{\partial L}{\partial a_{t+1}}$ is the gradient of the loss with respect to the hidden state at the next time step, which is backpropagated through time.**

**The gradient with respect to tanh is given by:**
**$\frac{\partial \tanh(a)} {\partial a}$**

**The gradients with respect to the parameters are then computed using the chain rule:**

**$\frac{\partial L}{\partial W_{ya}} = \sum_t \frac{\partial L}{\partial y_t} a_t$**

**$\frac{\partial L}{\partial b_y} = \sum_t \frac{\partial L}{\partial y_t}$**

**$\frac{\partial L}{\partial W_{ax}} = \sum_t \frac{\partial L}{\partial a_t} \frac{\partial a_t}{\partial W_{ax}}$**

**$\frac{\partial L}{\partial W_{aa}} = \sum_t \frac{\partial L}{\partial h_t} \frac{\partial h_t}{\partial W_{aa}}$**

**$\frac{\partial L}{\partial b_a} = \sum_t \frac{\partial L}{\partial a_t} \frac{\partial h_t}{\partial b_a}$**

**where $\frac{\partial h_t}{\partial W_{ax}}$, $\frac{\partial a_t}{\partial W_{aa}}$, and $\frac{\partial h_t}{\partial b_a}$ can be computed as:**

**$\frac{\partial a_t}{\partial W_{ax}} = x_t$**

**$\frac{\partial a_t}{\partial W_{aa}} = a_{t-1}$**

**$\frac{\partial a_t}{\partial b_a} = 1$**

**These gradients are then used to update the parameters of the RNN using an optimization algorithm such as gradient descent, Adagrad, or Adam.**
****

## Loss:

**The cross-entropy loss between the predicted probabilities y_pred and the true targets y_true at a single time step $t$ is:**

**$$H(y_{true,t}, y_{pred,t}) = -\sum_i y_{true,t,i} \log(y_{pred,t,i})$$**

**where $y_{pred,t}$ is the predicted probability distribution at time step $t$, $y_{true,t}$ is the true probability distribution at time step $t$ (i.e., a one-hot encoded vector representing the true target), and $i$ ranges over the vocabulary size.**

**The total loss is then computed as the sum of the cross-entropy losses over all time steps:**

**$$L = \sum_{t=1}^{T} H(y_{true,t}, y_{pred,t})$$**

**where $T$ is the sequence length.**

****


## Train:
**The train method trains the RNN on a dataset using backpropagation through time. The method takes an instance of DataReader containing the training data as input. The method initializes a hidden state vector a_prev at the beginning of each sequence to zero. It then iterates until the smooth loss is less than a threshold value.**

**During each iteration, it retrieves a batch of inputs and targets from the data reader. The RNN then performs a forward pass on the input sequence and computes the output probabilities. The backward pass is performed using the targets and output probabilities to calculate the gradients of the parameters of the network. The Adagrad algorithm is used to update the weights of the network.**

**The method then calculates and updates the loss using the updated weights. The previous hidden state is updated for the next batch. The method prints the progress every 500 iterations by generating a sample of text using the sample method and printing the loss.**


**The train method can be summarized by the following steps:**


**$1.$ Initialize $a_{prev}$ to zero at the beginning of each sequence.**

**$2.$ Retrieve a batch of inputs and targets from the data reader.**

**$3.$ Perform a forward pass on the input sequence and compute the output probabilities.**

**$4.$ Perform a backward pass using the targets and output probabilities to calculate the gradients of the parameters of the network.**

**$5.$ Use the Adagrad algorithm to update the weights of the network.**

**$6.$ Calculate and update the loss using the updated weights.**

**$7.$ Update the previous hidden state for the next batch.**

**$8.$ Print progress every 10000 iterations by generating a sample of text using the sample method and printing the loss.**

**$9.$ Repeat steps $2$-$8$ until the smooth loss is less than the threshold value.**

In [3]:
data_generator = DataGenerator('D:\Machine_Learning_From_Scratch\RNN\data\dinos.txt')

In [4]:
rnn = RNN(hidden_size = 200, data_generator = data_generator, sequence_length = 25, learning_rate = 1e-3)
rnn.train()



iter 0, loss 82.359704

Ffssyyjjrreehhddssrrbbxxssddvvwwxxddccttnnaaoobbffbbrrxx

Ccvvtt

Rrqqiilleejjccttjjjjhhsswwvvqqqqsskkkkaaaabboobbuunneejj

Ttffwwffnnxxggppddxxzzccvvccrrqqggnnhhzzssuu

Iiccbbnnddoorrggiizznnttlluuqqbbqqhhxxssrrxxddttppggtthhyyookkppttjjjjrrttgguuwwnnoossuuhhmmppnniiaa

iter 500, loss 58.355189

Aannrraatthhooppiiss



Ttiirreessaauuss

Eennyyllaappaauurruuss

Hheenneennyymmaaaa



iter 1000, loss 39.471550

Aaaarroommvvuuss

Lliissaauurruuss

Aaeeeelloossaauurruuss

Ddaappnnooppaaffrroommiimmuuss

Aaaarriimmppeessaauurruuss



iter 1500, loss 26.290693

Aabbrrooppyyoossaauuss

Aaaarrddoouussaauurruuss

Aaccaaiitthhoobbaattoorr

Nn

Aapprroossttaarruuss



iter 2000, loss 17.787749

Aaccaanntthhoopphhoolliiss

Aallrroossaauurruuss

Ddaassaauurruuss



Aaaaddaallllaahhssaauurruuss



iter 2500, loss 12.551106

Aammttiioossaauurruuss

Aaaattiilllluuss

Aammttiissaauurruuss

Hheeeeoonnhhrruuss

Aabbrroossaauurruuss



iter 3000, loss 9.326275

Aammeennyyxx

Aapp

In [5]:
rnn.predict('meo')

'meopisaurus'

In [6]:
rnn.predict('a')

'afroctobator'